### libarary

In [1]:
import pandas as pd
import numpy as np
import apiclient
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

### Global Apisettings

In [2]:
json_path = 'client_secrets.json' # fill json file's path

### Function

#### Google Analytics

In [4]:
def ga(view_id, m_list, d_list, start_date, end_date, filters):
    scope=['https://www.googleapis.com/auth/analytics.readonly']
    credentials=ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
    analytics = build('analyticsreporting', 'v4', credentials=credentials)
    df = pd.DataFrame(columns = d_list)
    metrics = [{'expression': 'ga:' + m} for m in m_list]
    dimensions = [{'name': 'ga:' + d} for d in d_list]
    page_size = 100000
    sampling_level = 'LARGE'
    
    while True: 
        body = {
            'reportRequests': [
                {
                    'viewId': view_id,
                    'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
                    'metrics': metrics,
                    'dimensions': dimensions,
                    'pageSize': page_size,
                    'filtersExpression': filters,
                    'samplingLevel': sampling_level
                }
            ]
        }

        response = analytics.reports().batchGet(body=body).execute()

        temp = pd.io.json.json_normalize(response['reports'][0]['data']['rows'])

        for i, d in enumerate(d_list):
            temp[d] = temp['dimensions'].apply(lambda x: x[i])

        for i, m in enumerate(m_list):
            temp[m] = temp['metrics'].apply(lambda x: x[0]['values'][i])

        temp.drop(columns=['dimensions', 'metrics'], inplace=True)
        df = df.append(temp) 
        page_token = response['reports'][0].get('nextPageToken')
        if page_token == None: break
    
    return df

#### Execution sample

**GA**

##### GA parameter

In [5]:
view_id = ''# fill view_id
m_list = ['sessions'] #check here https://developers.google.com/analytics/devguides/reporting/core/v4/rest/v4/reports/batchGet
d_list = ['landingPagePath','date']
start_date = '2019-08-15' # start fill date
end_date = '2019-08-15'# end fill date
filters = ''# fill conditions

In [6]:
gd_data = ga(view_id, m_list, d_list, start_date, end_date, filters)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
